In [39]:
# Necessary libraries
import mysql.connector as mysql
import pandas as pd 
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import sklearn
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from langdetect import detect
from  tqdm import tqdm
from datetime import datetime
from datetime import timedelta
import sqlalchemy
import pymysql
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ylee_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ylee_\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Obtaining Data

    The function below gets the keywords from the Keywords table from the database. The reason for this is to search the SRT21 table with each keyword in this table to extract text that contains the keywords from the SRT21 table.

In [68]:
## Getting keywords table from database

# enter your server IP address/domain name
HOST = "xxxxx" # or "domain.com"
# Database name, if you want just to connect to MySQL server, leave it empty
DATABASE = "xxxxxxx"
# This is the user you create
USER = "xxxxxxxxxxx"
# User password
PASSWORD = "xxxxxxxxxx"
# Connect to MySQL server
db_connection = mysql.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD)
print("Connected to:", db_connection.get_server_info())
# Fetch keywords
query_keywords = "SELECT * FROM xxxxxxxxxxx"
keywords = pd.read_sql_query(query_keywords, db_connection)

print("successful")
keywords

Connected to: 8.0.28-0ubuntu0.20.04.3
successful


,ID,Keyword,Submission_Time
0,1,mcdonalds,2022-03-05 18:58:19
1,2,walmart,2022-03-05 18:58:45


    The function below pulls the most recent day(i.e yesterday) of SRT21 table. The number of most recent days can be changed by altering the query below. Using pandas library, the data table is converted to a data frame so we can process the data.  

In [69]:
## Getting the most recent data

# Enter your server IP address/domain name
HOST = "xxxxxxxxxx" # or "domain.com"
# Database name, if you want just to connect to MySQL server, leave it empty
DATABASE = "xxxxxxx"
# This is the user you create
USER = "xxxxxxxxxx"
# User password
PASSWORD = "xxxxxxxxx"
# Connect to MySQL server
db_connection = mysql.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD)
print("Connected to:", db_connection.get_server_info())

# Query to fetch SRT21 table
querystring = "SELECT * FROM xxxxxxx where Created_at >= DATE_SUB(NOW(),INTERVAL 1 day)"
SRTtable = pd.read_sql_query(querystring, db_connection)
print("successful")

Connected to: 8.0.28-0ubuntu0.20.04.3
successful


In [70]:
# From None type to Dataframe
SRTtable = pd.DataFrame(SRTtable)

In [71]:
SRTtable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56685 entries, 0 to 56684
Data columns (total 7 columns):
Channel_Id       56685 non-null int64
Mp4_File_Name    56685 non-null object
CC_Num           56685 non-null int64
Timecode         56685 non-null object
Created_at       56685 non-null datetime64[ns]
Finished_at      56685 non-null datetime64[ns]
Cat_Frames       56685 non-null object
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 3.0+ MB


In [72]:
SRTtable['Cat_Frames']

0        -(bleats) -Axel! I knew you’d come back for me...
1        -Axel! I knew you’d come back for me. Oh, I lo...
2        with your wet slimy kisses. (screams) Ew, slug...
3        Hmm. Oh, right. I’ve been ditched, I’m lost, a...
4        Satanists. (grunting) Let us out! We’ll do Sat...
                               ...                        
56680    ARRIBA EL EQUIPO DE  CONTENDIENTES APROVECHAND...
56681    DE AYER.. PARA ALFREDO. UNOS 56 1 A 0 LA PIZAR...
56682    EQUIPO DE CONTENDIENTES A  CONTINUACIóN LA SEG...
56683    CONCENTRADAS LISTAS 3 2 1 BAJA LA BARRA DE INI...
56684    PIE DERECHO EL EQUIPO DE LOS  CONTENDIENTES ES...
Name: Cat_Frames, Length: 56685, dtype: object

## Data Cleaning
    The data cleaning phase involves removing missing data, converting letters to lower-case letters only, removing punctuation, and keeping only English text.

In [73]:
# Function to remove punctuation and stopwords
def process(text):
    
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]  # Remove stopwords
    words = ""
    for i in text:  #word stemming
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

In [74]:
# Code to remove non-ascii characters. 
def cleandata(text):
    
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
   
    # Remove any non ASCII characters
    encoded = text.encode('ascii', errors='ignore')
    decode = encoded.decode()
   
    return decode

In [75]:
# Applying the data cleaning functions
SRTtable['Cat_Frames'] = SRTtable['Cat_Frames'].fillna(' ')
SRTtable['Cat_Frames'] = SRTtable['Cat_Frames'].apply(process)
SRTtable['Cat_Frames'] = SRTtable['Cat_Frames'].apply(cleandata)

In [76]:
# Checking which lines throws errors because they're empty
text = SRTtable['Cat_Frames']
langdet = []                                       
indexError = []

for i in range(len(SRTtable)):
    try:
        lang = detect(text[i])
        langdet.append(lang)
    except:
        indexError.append(i)

In [77]:
# Drop index with errors
for i in indexError:
    SRTtable = SRTtable.drop(i, axis=0)

In [78]:
# Keeping the ones that are English since we are only working with English for now
SRTtable['is_english'] = SRTtable['Cat_Frames'].apply(lambda x: detect(x) == 'en')
SRTtable = SRTtable[SRTtable.is_english == True]

In [79]:
SRTtable['Cat_Frames']=SRTtable['Cat_Frames'].astype(str)

In [80]:
SRTtable['Cat_Frames']

0             bleat axel knew youd come back oh love goat 
1        axel knew youd come back oh love goat wet slim...
2        wet slimi kiss scream ew slug ew ew ew ew ew e...
3        hmm oh right iv ditch im lost peopl look satan...
4        satanist grunt let us well satan stuff like gr...
                               ...                        
56605    album jason thrill fan iheart radio theater re...
56606    macon georgia it alway fun play especi new son...
56607    set set song apart think feel it one big recor...
56608    it one big record 20 new song ail bull ten liv...
56609    never put greatest hit record live album anyth...
Name: Cat_Frames, Length: 40569, dtype: object

## Text preparation
    After cleaning the data, we need to prepare the data for the sentiment analysis. That means in order to get text related to the key word, we first need to group the data by channel because text in context can only be from one channel. Moreover, the SRT21 table's Cat_Frames column has overlapping text which means half the text in row 1 repeats in row 2 and so on. The reason for it is to ensure that data leaking does not happen. The overlapping causes a dublication in data, which has to be removed to continue getting context from text and getting more accurate sentiment results.  

In [81]:
# Function to make the Cat Frames column into a list
def createList(conv_df):
    cclist = []
    # Iterate over each row
    for index, rows in conv_df.iterrows():
        # Create list for the current row
        my_list =[rows.Cat_Frames]
        # append the list to the final list
        cclist.append(my_list)
        
    print(cclist[:10])
    
    return cclist

In [82]:
# Function that takes in the new list and removes the overlapping text in each row
def removeOverlap(cclist):
    
    try:
        for sentence in range(len(cclist)):
            # Set the two strings to elements in the list.
            string1 = cclist[sentence]
            string2 = cclist[sentence + 1]

            # If the strings are a perfect match, drop one.
            if string1 == string2:
                cclist.remove(string2)

            else:
                # Convert to type string.
                string1 = ' '.join(string1)
                string2 = ' '.join(string2)

                # Split into a word list.
                str1array = string1.split()
                str2array = string2.split()

                # Find the the last word of string1 in string2 and note the index.
                # Use that index to make a new string2.
                if str1array[-1] in str2array:
                    index = str2array.index(str1array[-1])
                    string2 = str2array[index + 1:]
                    string2 = ' '.join(string2)
                    cclist[sentence + 1] = string2
                else:
                    continue

    # For when the index is out of range.
    except IndexError:
        print("end of list")
        
        return cclist

In [83]:
# Function to convert the list back into the Cat Frames column
def replaceCatFrames(conv_df, cclist):
    
    conv_df['Cat_Frames'] = cclist
    
    return conv_df

In [84]:
# Remove the overlapping text
cclist = createList(SRTtable)
finalTextList = removeOverlap(cclist)
SRTtable = replaceCatFrames(SRTtable, finalTextList)

[['bleat axel knew youd come back oh love goat '], ['axel knew youd come back oh love goat wet slimi kiss scream ew slug ew ew ew ew ew ew '], ['wet slimi kiss scream ew slug ew ew ew ew ew ew hmm oh right iv ditch im lost peopl look '], ['hmm oh right iv ditch im lost peopl look satanist grunt let us well satan stuff '], ['satanist grunt let us well satan stuff like grow pointi beard draw eyelin ask final time sacrific '], ['draw eyelin ask final time sacrific told dont know goat prepar die '], ['grunt yawn time happen oh sweeti got littl overexcit barf sleep wait spent night sleep bed '], ['center spoon doesnt get child wait there still time us genderblind product crucibl whatev word mean meh that kind jule thing right id kind rather stay bed '], ['whatev word mean meh that kind jule thing right id kind rather stay bed lisa three mm celtic heavi metal music play '], ['lisa three mm celtic heavi metal music play dont need stupid martin make ankl rock compass way go ']]
end of list


In [85]:
print(finalTextList[:10])

[['bleat axel knew youd come back oh love goat '], 'wet slimi kiss scream ew slug ew ew ew ew ew ew', ['wet slimi kiss scream ew slug ew ew ew ew ew ew hmm oh right iv ditch im lost peopl look '], 'satanist grunt let us well satan stuff', ['satanist grunt let us well satan stuff like grow pointi beard draw eyelin ask final time sacrific '], 'told dont know goat prepar die', ['grunt yawn time happen oh sweeti got littl overexcit barf sleep wait spent night sleep bed '], '', ['whatev word mean meh that kind jule thing right id kind rather stay bed lisa three mm celtic heavi metal music play '], ['lisa three mm celtic heavi metal music play dont need stupid martin make ankl rock compass way go ']]


In [86]:
SRTtable['Cat_Frames']=SRTtable['Cat_Frames'].astype(str)

In [87]:
SRTtable = SRTtable.drop_duplicates(subset=['Cat_Frames'])

In [88]:
SRTtable['Cat_Frames'] = SRTtable['Cat_Frames'].apply(process)

In [89]:
# Grouping the data by channel
grouped_Channel = SRTtable.groupby(SRTtable.Channel_Id)
Channel_ID_List = SRTtable['Channel_Id'].unique()
keywordList = keywords['Keyword'].tolist()

## Get Conversation function
    This function gets keyword-related rows from SRT21 table. Because of text that is about a keyword, it is not nassesary to contain the keyword. Also, getting only rows contain the keyword is inefficient because we can not retrive enough data to perform the sentiment analysis. Therefore, we found an approach to get as much related rows by searching for keyword in rows. Then see the time gap between two keyword. If there is another hit of the keyword in less than 2 minutes, we cosider all rows between these two hits of the keyword as one conversation. On the other hand, if only one hit of the keyword in the two minutes length, we only consider 30 seconds length as a conversation. Programmically, the function takes data frame from SRT21 table with only one channel data in it, then split timecode column into start time and end time columns to use it later on to check times between keyword hits. Then, it search for all hit in the dataframe and put it in a new dataframe. Then, it goes through all hits in the new dataframe and check for times between hits, if the time is more than 2 minutes, it will get the previous and next row of the hit row from the original dataframe using indexes. Also, if the time between two hits is less than two minutes, it will consider all text between these two hits are in same context and it will get all rows in between to the new dataframe. Surely, the accuracy of capturing the conversaion would not be great. Meaning that some in-context would not be captured, and some out-of-context text would be include in "conversation" text. To increase the accuracy and improve the detecting conversations, Machine Learning algorarithms must be involved to recognize start and end of a conversation by learning when a conversation or a dialoage would start and conclude.  

In [95]:
# Getting keyword hits and needed rows that may related to the keywords(i.e a conversation)
# Df is dataframe of one channel 
def get_conversation(df):
    
    # Split the two times in timecode column into their own columns
    df[['StartTime', 'EndTime']] = df['Timecode'].str.split(',', expand=True)
    # Create index column
    df = df.reset_index()
    conv_id = 0
    # Get keyword and search for it in the df
    # Put keyword hits into a new df
    for keyword in keywordList:
        for row, index in df.iterrows():
            newdf = df[df['Cat_Frames'].str.contains(keyword)]
            newdf['keyword'] = keyword
           
        for row, index in newdf.iterrows():
            # Set format of the timecode column
            date_format_str = '%H:%M:%S.%f'
            minutes = 2

            for index in range(len(newdf)):
                newdf['Conversation_ID'] = conv_id + 1
                # Initialize variables for the times
                try:
                    nextIndex = index + 1

                    initial_HitStart = newdf.loc[newdf.index[index], 'StartTime']
                    initial_HitEnd = newdf.loc[newdf.index[index], 'EndTime']

                    next_HitStart = newdf.loc[newdf.index[nextIndex], 'StartTime']
                    next_HitEnd = newdf.loc[newdf.index[nextIndex], 'EndTime']

                    # Set to time format
                    initial_start_time = datetime.strptime(initial_HitStart, date_format_str)
                    initial_end_time = datetime.strptime(initial_HitEnd, date_format_str)

                    next_start_time = datetime.strptime(next_HitStart, date_format_str)
                    next_end_time = datetime.strptime(next_HitEnd, date_format_str)

                    # Add 2 minutes to the time
                    final_initialStart_time = initial_start_time + timedelta(minutes = minutes)
                    final_initialEnd_time = initial_end_time + timedelta(minutes = minutes)

                    # Check if the time of the next hit is 2 minutes after the first hit ends
                    if next_start_time == final_initialEnd_time:
                        # Check for the rows' index in the old dataframe
                        # Add indices to a list
                        location = []
                        location.append(newdf.loc[newdf.index[index], 'index'])
                        location.append(newdf.loc[newdf.index[nextIndex], 'index'])
                        index_range = range(location[0] + 1, location[1])
                        index_list = list(index_range)
                        # Add inbetween rows to new dataframe
                        for position in index_list:
                            try:
                                newdf.append(df.iloc[position])
                            except (ValueError, IndexError):
                                continue

                    # If the rows are more than 2 minutes apart, do the same process but get the row above and below the initial row only
                    else:
                        location = []
                        location.append(newdf.loc[newdf.index[index], 'index'])
                        for position in location:
                            try:
                                newdf.append(df.iloc[position + 1])
                                newdf.append(df.iloc[position - 1])
                            except (ValueError, IndexError):
                                continue
                except (ValueError, IndexError):
                    continue

    return newdf

## Sentiment Analysis function

    The function below takes data drame with only text in context that associated to a keyword and apply sentiment test. The sentiment analysis contains two type of test. The first test called polarity test which gives a score of how positive or negative is the text. Moreover, it contains 3 scores from 0 to 1 a score for negativity of the text, the neturality of the text, and the positivity of the text. The other test called subjectivity test, it is test the text and gives a score for that tells if the text is based on opinion or facts, it test the language used in the text.

In [92]:
def sentiment_analysis(final_df):
    report = pd.DataFrame()
    report['keyword'] = final_df['keyword'].astype(str)
    report['Conversation_ID'] = 1
    report['Channel_Id'] = final_df['Channel_Id'].astype(str)
    report['Mp4_File_Name'] = final_df['Mp4_File_Name'].astype(str)
    report['CC_Num'] = final_df['CC_Num'].astype(str)
    report['Timecode'] = final_df['Timecode'].astype(str)
    report['Created_at'] = final_df['Created_at'].astype(str)
    report['Finished_at'] = final_df['Finished_at'].astype(str)

    sid = SentimentIntensityAnalyzer()
    
    report['polarity_scores'] = final_df['Cat_Frames'].apply(lambda Cat_Frames: sid.polarity_scores(Cat_Frames))
    report['compound'] = final_df['Cat_Frames'].apply(lambda Cat_Frames: sid.polarity_scores(Cat_Frames)['compound'])
    report['comp_score'] = report['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')
    report['SubjScores'] = final_df['Cat_Frames'].apply(lambda Cat_Frames: TextBlob(Cat_Frames).sentiment.subjectivity)
    report['Subjectivity'] = report['SubjScores'].apply(lambda c: 'Subjective' if c >=0.5 else 'Objective')

    report['polarity_scores'] = report['polarity_scores'].astype(str)
    report['compound'] = report['compound'].astype(str)
    report['SubjScores'] = report['SubjScores'].astype(str)
    report['comp_score'] = report['comp_score'].astype(str)
    report['Subjectivity'] = report['Subjectivity'].astype(str)

    report['Conversation_ID'] = report['Conversation_ID'].astype(str)

    return report

## Writing the results to the database 

In [1]:
def write_to_db(df):
    database_username = 'xxxxxxxxx'
    database_password = 'xxxxxxxx'
    database_ip       = 'xxxxxxxx'
    database_name     = 'xxxxxxxx'
    database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name), pool_recycle=1, pool_timeout=57600).connect()
    df.to_sql(con=database_connection, name='xxxxxxxxx', if_exists='append',index= False)
    database_connection.close()
    

## Here where the magic happens

    After Cleaning phase and grouping the clean data by Channels, the goal is sending every channel dataframe to get_conversation function and then perform the sentiment analysis and write the result back to the database. 

In [96]:
# looping through channels to create dataframes for each channel and send each dataframe to get_conversation function
# conv_df supposed to contain only data frame of desired conversation
count = 0
for i in range(len(Channel_ID_List)):
    ## get df of one channel as df
    df = grouped_Channel.get_group(Channel_ID_List[i])
    ## get conversation from channel df
    conv_df = get_conversation(df)   
    ## perform sentiment analysis on conversation df
    report_df = sentiment_analysis(conv_df)
    # write result to db
    write_to_db(report_df)

C:\Users\ylee_\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\ylee_\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Conclusion

To conclude, this project obtain, clean, and extraact conversaions from TV recoreded subtitles. on personal computers, this file takes around 50 minutes to run SRT21 table for one day. We recommend to improve capturing the context of text by investigating how to indicate conversations' start and end. Also, it is possible to use supervised machile leaning algorathims to automate the process. by feeding the model with labling text with thier keyword to train the model.  


At the end, we, Yvonne and Ahmad, thank Digiclips and Christy Pearson for the opprtonity to work on a real-life project. This project gave us new skills and taought us new frontiers in data Science. We wish Digiclips the best in thier business and we hope that we were helpful. Also, we apreciate Ms. Pearson for her guidnece and support throughout the semester.